In [1]:
import tensorflow as tf
from tensorflow import image
from tensorflow.keras import layers, models, preprocessing
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import cv2
print(tf.__version__)

# this stuff should probs go into the driver script
# only using the non-tf packages for the preprocessing of data
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
import pandas as pd
import wget

# Want to try and download the dataset from URL and unzip
url = "https://cloudstor.aarnet.edu.au/sender/download.php?token=d82346d9-f3ca-48bf-825f-327a622bfaca&files_ids=9881639"



In [3]:
wget.download(url)

  0% [                                                                    ]    1286144 / 1698475197

  0% [                                                                    ]    2293760 / 1698475197

  0% [                                                                    ]    3637248 / 1698475197

  0% [                                                                    ]    5087232 / 1698475197

  0% [                                                                    ]    6529024 / 1698475197

...

  1% [                                                                    ]   24756224 / 1698475197

  1% [.                                                                   ]   26058752 / 1698475197

  1% [.                                                                   ]   27353088 / 1698475197

100% [....................................................................] 1698475197 / 1698475197

'AKOA_Analysis.zip'

In [ ]:
import zipfile
with zipfile.ZipFile("AKOA_Analysis.zip", "r") as zip_ref:
    zip_ref.extractall()
# file now in directory called AKOA_Analysis, that contains all the images

Need to move all the data processing stuff into this file. THen figure out how to call the model.

In [2]:
# this stuff should probs go into the driver script
import random

data_dir = "AKOA_Analysis"

data_dir = pathlib.Path(data_dir)

print('total images', len(list(data_dir.glob('*.png'))))

left_images = list(data_dir.glob('*left*.png'))
left_underscored = list(data_dir.glob('*L_E_F_T*.png'))
left_images = left_images + left_underscored
print('left images', len(left_images))

right_images = list(data_dir.glob('*right*.png'))
right_underscored = list(data_dir.glob('*R_I_G_H_T*.png'))
right_images = right_images + right_underscored
print('right count', len(right_images))

print('sum ', len(right_images)+len(left_images))

total images 18680
left images 7760
right count 10920
sum  18680


In [ ]:
# this stuff should probs go into the driver script
# Randomising the collection of images

total_images = list(data_dir.glob('*.png'))
print('total images first', total_images[1])
random.seed(87)
random.shuffle(total_images)
print('total images first post shuffle', total_images[1])
#total_images = total_images[1:1000]

# want to extract the labels for 
# Need to take a selection for training, testing and validation 

In [ ]:
# this stuff should probs go into the driver script
total_images_labels =np.array([None]*len(total_images))
left = ["left", "l_e_f_t"]
right = ["right", "r_i_g_h_t"]

for i in range(len(total_images)):
    path = total_images[i]
    # make path names all lowercase
    path = str(path).lower()
    #total_images_labels = total_images_labels.reshape((10023,1))
    # give labels for eahc case 
    if any(x in path for x in left):
        total_images_labels[i] =0 #left
    elif any(x in path for x in right):
        total_images_labels[i] =1 #right
    else:
        print(total_images[i]) # print to screen any path names that aren't covered
total_images_labels = total_images_labels.reshape((len(total_images),1))
#total_labels = total_images_labels.reshape((10023,1))
#print(total_images_labels)
#print(total_images[8])
print(total_images_labels.shape)

In [ ]:
# this stuff should probs go into the driver script
# read in all the images to the list 

# might be better to have this as an iterator/generator: checkout Assignments/Prac Notes/Coding Exercise
loaded_images = np.array([None]*len(total_images))
#loaded_images = tf.Variable(0)
for i in range(len(total_images)):
    #loade_images[i] = tf.image.resize(total_images[i])
    #loaded_images[i] = plt.imread(total_images[i]).reshape((1, 228, 260, 3))
    
    #total_images[i]= plt.imread(total_images[i])
    #new
    #loaded_images.append(img_data)
    loaded_images[i] = cv2.resize(plt.imread(total_images[i]),(64,64)).reshape((1,64,64,3)) #may want to change this size


In [ ]:
# this stuff should probs go into the driver script
def conv_tensor(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float32)
    return arg

#fixing the shape of the images data
print(loaded_images.shape)
print(loaded_images[1].shape)
#for i in range(len(loaded_images)):
#loaded_images_2 = loaded_images.reshape((10023, 228, 260, 3))
loaded_images =np.concatenate(loaded_images, axis=0)
#loaded_images = conv_tensor(loaded_images)
print(loaded_images.shape)
#print(loaded_images[5])

In [ ]:
# this stuff should probs go into the driver script


train_images, test_images, train_labels, test_labels = train_test_split(loaded_images,
                total_images_labels,test_size=0.25, random_state=101)
print(len(train_images))
print(len(train_labels))
print(len(test_images))
print(len(test_labels))

sum_train_left=0
sum_train_right=0
sum_test_left=0
sum_test_right=0
for i in range(len(train_labels)):
    # make path names all lowercase
    if train_labels[i] ==0:
        sum_train_left +=1
    elif train_labels[i] ==1:
        sum_train_right +=1
for i in range(len(test_labels)):   
    if test_labels[i]==0:
        sum_test_left +=1
    elif test_labels[i]==1:
        sum_test_right+=1

print('Sum left train', sum_train_left)
print('sum right train', sum_train_right)
print('Sum left test', sum_test_left)
print('sum right test', sum_test_right) 

plt.imshow(train_images[4])
print(train_labels[4])

train_images = conv_tensor(train_images)
test_images = conv_tensor(test_images)
train_labels = conv_tensor(train_labels)
test_labels = conv_tensor(test_labels)

#plt.figure(figsize=(5,5))
#print(img1.shape)
# TODO I don't think the stuff is in the right format yet. 
#Might need to reshape it so in form e.g. images: (10023, 228, 260,3), labels: (10023, 1)
#plt.imshow(train_images[4])
#plt.xlabel(tf.strings.as_string(train_labels[4], precision=0.0))
#print(train_images[4].shape)
#tf.print(train_labels[4])
#plt.imshow(test_images[4])
#plt.xlabel(tf.print(test_labels[4]))


In [2]:
# Need to call the function in the model script
from Classify-Knee-Laterality-Model.ipynb import *

SyntaxError: invalid syntax (<ipython-input-2-0aa2bb7d8715>, line 2)

In [ ]:


# evaluate and plot the stuff

#this below stuff may end up going into the driver script ??
model.summary()
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics =['accuracy'])

#fit the keras model to the dataset
history=model.fit(train_images,train_labels,batch_size=32, epochs=10, validation_data=(test_images, test_labels))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)